In [1]:
import copy
import numpy as np
import open3d as o3
from probreg import cpd
from probreg import gmmtree
from probreg import filterreg
from timeit import default_timer as timer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# # load source and target point cloud
# source = o3.io.read_point_cloud('bunny.pcd')
# target = copy.deepcopy(source)

In [3]:
# load source and target point cloud
mesh = o3.io.read_triangle_mesh('test.obj')
source = mesh.sample_points_uniformly(2048)
target = copy.deepcopy(source)

In [4]:
# transform target point cloud
th = np.deg2rad(30.0)
target.transform(np.array([[np.cos(th), -np.sin(th), 0.0, 0.0],
                           [np.sin(th), np.cos(th), 0.0, 0.0],
                           [0.0, 0.0, 1.0, 0.0],
                           [0.0, 0.0, 0.0, 1.0]]))
source = source.voxel_down_sample(voxel_size=0.005)
target = target.voxel_down_sample(voxel_size=0.005)

In [5]:
# compute cpd registration

start = timer()

#tf_param, _, _ = cpd.registration_cpd(source, target)
#tf_param, _ = gmmtree.registration_gmmtree(source, target, maxiter=150)
tf_param, _, _ = filterreg.registration_filterreg(source, target)
result = copy.deepcopy(source)
result.points = tf_param.transform(result.points)

end = timer()

print("Total tunning time: {:4f}".format(end - start))

Total tunning time: 0.100229


In [8]:
# draw result
source.paint_uniform_color([1, 0, 0])
target.paint_uniform_color([0, 1, 0])
result.paint_uniform_color([0, 0, 1])
o3.visualization.draw_geometries([source, target, result])